In [ ]:
!pip3 install pika

In [1]:
import pika

Тут мы объявляем метод для обработки полученного сообщения и для получения сообщений.
Не забудьте указать корректные логин, пароль и корректное название очереди

In [13]:
import json

def process_message(ch, method, properties, body):
    try:
        # Парсинг JSON-сообщения
        message = json.loads(body)
        print(f"Получено сообщение: {message}")

        # Пример обработки сообщения
        if 'service' in message:
            print(f"Обработка данных для сервиса: {message['service']}")
    except json.JSONDecodeError:
        print("Ошибка: Невозможно распознать сообщение как JSON")
    finally:
        # Подтверждаем получение сообщения
        ch.basic_ack(delivery_tag=method.delivery_tag)



def receive_message():
    # Учетные данные для авторизации
    credentials = pika.PlainCredentials('user', 'password')  # Используйте вашего пользователя и пароль
    connection_params = pika.ConnectionParameters(
        host='localhost',       # Хост кластера
        port=5672,              # Порт AMQP
        credentials=credentials # Учетные данные для подключения
    )

    # Устанавливаем соединение с RabbitMQ
    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()

    # Указываем, что будем получать сообщения из очереди 'test_queue'
    channel.basic_consume(
        queue='test_queue',
        on_message_callback=process_message
    )

    print("Ожидание сообщений. Нажмите Ctrl+C для завершения.")
    # Начинаем приём сообщений
    channel.start_consuming()

А здесь включаем сам консьюмер

In [ ]:
receive_message()

Ожидание сообщений. Нажмите Ctrl+C для завершения.
Получено сообщение: {'id': 1382, 'message_value': 123, 'message_prop': 'dd', 'service': 'basic-service'}
Обработка данных для сервиса: basic-service
Получено сообщение: {'id': 2266, 'message_value': 123, 'message_prop': 'dd', 'service': 'basic-service'}
Обработка данных для сервиса: basic-service
Получено сообщение: {'type': 'order.physical', 'id': '71be5354-a009-474d-938a-0432119a82fa', 'name': 'Хохлов Виктор Денисович', 'items': [{'item_id': 6746261, 'quantity': 14}, {'item_id': 8764928, 'quantity': 13}], 'inn': '377274612071'}
Получено сообщение: {'type': 'question', 'id': 'ca277bde-376c-4ddc-a031-da4315449514', 'name': 'Овчинникова Прасковья Рудольфовна', 'theme': 'payment', 'question': 'Спасть манера вскинуть солнце печатать иной заявление совещание близко место солнце призыв школьный.'}
Получено сообщение: {'type': 'order.legal', 'id': '42cb6325-01eb-473a-98a7-8fd6d2e60311', 'name': 'Макар Фомич Герасимов', 'items': [{'item_id': 